In [ ]:
dict(map(lambda x, y:(x, y), zip("abcde", range(500))))  # ？会报错吗

# 柯里化Curring

In [1]:
# 柯里化：将原来接受两个参数的函数变成接受一个参数的函数的过程。新的函数返回一个以原第二参数为参数的函数
# 例：f(x, y) -> f(x)(y)

# ！装饰器

In [12]:
def copy_properties(src, dest):
    dest.__name__ = src.__name__
    dest.__doc__ = src.__doc__
    
def copy_properties(src):
    def _wrapper(dest):
        dest.__name__ = src.__name__
        dest.__doc__ = src.__doc__
        return dest
    return _wrapper
    

def logger(duration, func=lambda name, duration: print(name, duration)):  # 柯里化保持三层，就足够
    def _logger(fn):
        # @functools._wrapper(fn)  # _wrapper == functools._wrapper(fn)(_wrapper)
        @copy_properties(fn)  # _wrapper == copy_properties(fn)(_wrapper)  fn对应装饰器中参数dest
        def _wrapper(*args, **kwargs):
            print("begin")
            res = fn(*args, **kwargs)
            print("end")
            print("*" * 30)
            func(fn.__name__, duration)
            print("*" * 30)
            return res
        # functools.update_warpper(_wrapper, fn)  # 系统自带功能
        # copy_properties(fn, _wrapper)  # 重写装饰器的属性，和被包装函数一致
        return _wrapper
    return _logger

@logger(5)  # (logger(5) == _logger) => (logger(5)(minus) == _logger(minus))
def minus(x, y):
    """This is a minus function
    
    for example ...
    """
    return x - y

print(minus(3, 4), minus.__name__, minus.__doc__)

begin
end
******************************
minus 5
******************************
-1 minus This is a minus function
    
    for example ...
    


# functools模块

In [ ]:
functools.update_wrapper(wrapper, wrapped, assigned=WRAPPER_ASSIGNMENTS, updated=WRAPPER_UPDATES)
# C:\\Users\\Administrator\\Anaconda3\\lib\\functools.py

# 函数注解

In [14]:
# pycharm: Insert documentation string stub
def add(x:int, y):
    """

    :param x: int
    :param y: int
    :return: int
    """
    return x + y


print(type(add.__annotations__), add.__annotations__)
print(type(add.__annotations__["x"]), add.__annotations__["x"])
# print(type(add.__annotations__["y"]), add.__annotations__["y"])  # KeyError: 'y'

<class 'dict'> {'x': <class 'int'>}
<class 'type'> <class 'int'>


In [15]:
i:int = 6  # 函数类型注解，非强制性的
i = "abc"  # 依然可以用str，动态语言的性质未变
i

'abc'

# inspect模块

In [26]:
from inspect import signature

def foo(a, *, b:int, **kwargs):  # 可变参数后面不要加:int，可以加不会报错
    pass

sig = signature(foo)
sig

<Signature (a, *, b: int, **kwargs)>

In [28]:
print(type(sig), sig)  # print函数会默认把参数转为str == print(str(sig))
print(sig.parameters)  # OrderedDict
print(sig.parameters['b'].annotation, sig.parameters['kwargs'].annotation)  # inspect._empty != None，模块下的空值

<class 'inspect.Signature'> (a, *, b: int, **kwargs)
OrderedDict([('a', <Parameter "a">), ('b', <Parameter "b: int">), ('kwargs', <Parameter "**kwargs">)])
<class 'int'> <class 'inspect._empty'>


# partial/偏函数

In [ ]:
# 把一个函数的部分参数固定下来，返回一个新的函数，新函数是对原函数的封装。

In [4]:
from functools import partial
import inspect

def add(x, y, z):
    return x + y * z

newadd = partial(add, y=4)
print(inspect.signature(newadd))
print(newadd(5, z=4))  # z变成了keyword-only参数

(x, *, y=4, z)
21


In [5]:
def add(x ,y):
    return x + y

# partial的部分原码
def partial(func, *args, **kwargs):
    def newfunc(*fargs, **fkwargs):
        newkwargs = kwargs.copy()
        newkwargs.update(fkwargs)
        return func(*(args + fargs), **newkwargs)
    
    newfunc.func = func  # 对原有函数的包装
    newfunc.args = args  # 空元组
    newfunc.kwargs = kwargs  # {"y":6}
    return newfunc  # 返回一个新的函数

newadd = partial(add, y=6)  #  newadd = newfunc
newadd(4)

10

# lru_cache

In [ ]:
# 使用前提：同样的函数参数一定得到同样的结果
#           函数执行时间很长，且要多次执行
# 缺点：
#     不支持缓存过期
#     不支持清除操作
#     不支持分布式，是一个单机的缓存
# 使用场景：
#    单机上需要空间换时间的地方

# 理解，实用价值不高

In [ ]:
# lru:Least-recently-used装饰器
@functool.lru_cache(maxsize=128, typed=False)

In [7]:
import functools
import time

def add(x, y, z=5):
    time.sleep(z)
    return x + y

add(4, 6)  # 1
add(4.0, 6)  # 1
add(4, y=6)  # 2
add(x=4, y=6)  # 3
add(y=6, x=4)  # 3:字典是经过排序处理的
# 根据args，kwargs是否相同，觉得是否缓存

10

In [8]:
import functools

@functools.lru_cache(maxsize=200)  # 添加缓存，效率的差异
def fib(n):
    if n < 3:
        return 1
    return fib(n-1) + fib(n-2)

print([fib(i) for i in range(35)])

[1, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]
